# Multiple Devices

In [2]:
import torch

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"Number of CUDA devices available: {device_count}")
    for i in range(device_count):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available on this system.")

Number of CUDA devices available: 2
Device 0: NVIDIA RTX A6000
Device 1: NVIDIA RTX A6000


# It is too slow!
## [02:03,  1.24s/it]

In [4]:
import time
import os
start = time.time()

train_epochs=20
learning_rate=0.001
llama_layers=32

# num_process=1
batch_size=12
d_model=16
d_ff=64
# --num_processes 1 --num_machines 1
comment='TimeLLM-FR'

os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"


!python -m accelerate.commands.launch  --multi-gpu --mixed_precision bf16 --num_processes=2 --num_machines 1 --dynamo_backend "no" --main_process_port "01025" ./Time-LLM/run_main.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./datasets/ \
  --data_path FR_data.csv \
  --model_id 1 \
  --model "TimeLLM" \
  --data FR \
  --features M \
  --seq_len 512 \
  --label_len 48 \
  --pred_len 96 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1 \
  --llm_model "LLAMA" \
  --llm_dim 4096 \
  --percent 20 \
  --d_model $d_model \
  --d_ff $d_ff \
  --batch_size $batch_size \
  --learning_rate $learning_rate \
  --llm_layers $llama_layers \
  --train_epochs $train_epochs \
  --model_comment $comment

end = time.time()

print("Total time:", (end - start)/60, "min.")

[2024-05-19 19:26:32,848] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-05-19 19:26:32,849] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-05-19 19:26:33,899] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-05-19 19:26:33,899] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-05-19 19:26:33,900] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
train 16323
val 19227
test 19155
train 16323
val 19227
test 19155
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:23<00:00, 11.93s/it]
Hourly data detailing load (electricity consumption), solar generation, and wind generation. These metrics are crucial in the electric power demand planning. 
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:24<00:00, 12.27s/it]
Hourly data detailing load (electricity consumption), solar generation, and wind generation. These metrics

# 41it [00:51,  1.42s/it]

In [6]:
import time
import os
start = time.time()

train_epochs=20
learning_rate=0.001
llama_layers=32

# num_process=1
batch_size=12
d_model=16
d_ff=64
# --num_processes 1 --num_machines 1
comment='TimeLLM-FR'

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


!python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --num_machines 1 --dynamo_backend "no" --main_process_port "01025" ./Time-LLM/run_main.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./datasets/ \
  --data_path FR_data.csv \
  --model_id 1 \
  --model "TimeLLM" \
  --data FR \
  --features M \
  --seq_len 512 \
  --label_len 48 \
  --pred_len 96 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1 \
  --llm_model "LLAMA" \
  --llm_dim 4096 \
  --percent 20 \
  --d_model $d_model \
  --d_ff $d_ff \
  --batch_size $batch_size \
  --learning_rate $learning_rate \
  --llm_layers $llama_layers \
  --train_epochs $train_epochs \
  --model_comment $comment

end = time.time()

print("Total time:", (end - start)/60, "min.")

train 16323
val 19227
test 19155
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:04<00:00,  2.09s/it]
Hourly data detailing load (electricity consumption), solar generation, and wind generation. These metrics are crucial in the electric power demand planning. 
[2024-05-19 19:34:33,047] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-05-19 19:34:33,995] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-05-19 19:34:33,995] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-05-19 19:34:33,995] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2024-05-19 19:34:34,903] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=141.20.21.42, master_port=29500
[2024-05-19 19:34:34,903] [INFO] [comm.py:668:init_distributed] Initializing

In [7]:
print("CUDA version:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("Number of GPUs:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

CUDA version: 12.1
CUDA available: True
Number of GPUs: 2
GPU 0: NVIDIA RTX A6000
GPU 1: NVIDIA RTX A6000


# 2.94s/it slower than with Accelerate

In [18]:
import time
import os
start = time.time()

train_epochs=20
#learning_rate=0.0001
# learning_rate= 0.01
# Epoch: 1 | Train Loss: 0.3189694 Vali Loss: 0.3789306 Test Loss: 0.4752660 MAE Loss: 0.4676420
learning_rate = 0.001
llama_layers=32 # 626 sec on 1 epoch # 13 min, # 1095 sec

# num_process=1
batch_size=8
d_model=16
d_ff=64
# --num_processes 1 --num_machines 1
comment='TimeLLM-FR'
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Set the desired CUDA device
# device = torch.device('cuda:1')
#os.environ["TORCH_USE_CUDA_DSA"] = "1"

# Set the current CUDA device
# torch.cuda.set_device(device)

!python -u ./Time-LLM/run_main_onecycle_no_acc.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./datasets/ \
  --data_path FR_data.csv \
  --model_id FR_96_24 \
  --model "TimeLLM" \
  --data FR \
  --features M \
  --seq_len 512 \
  --label_len 48 \
  --pred_len 96 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1 \
  --llm_model "LLAMA" \
  --llm_dim 4096 \
  --d_model $d_model \
  --percent 20 \
  --d_ff $d_ff \
  --batch_size $batch_size \
  --learning_rate $learning_rate \
  --llm_layers $llama_layers \
  --train_epochs $train_epochs \
  --model_comment $comment

end = time.time()

print("Total time:", (end - start)/60, "min.")

train 16323
val 19227
test 19155
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:01<00:00,  1.76it/s]
d_llm 4096
87it [04:17,  2.99s/it]^C
Total time: 5.121292042732239 min.


In [13]:
torch.cuda.is_available()
torch.cuda.current_device()
torch.cuda.device_count()

2